In [1]:
from config import config, load_abis
from web3 import Web3
from decimal import Decimal

from uniswap_pool_helper import UniswapPoolHelper, Token

In [2]:
# -- Test Configuration (using a WETH/USDC pool on Sepolia) --
# You can find other pools on the Uniswap Info site.
pool_address = "0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1" # WETH/USDC 0.05% on Sepolia

# 1. Initialize Web3 and load ABIs
print("--- Initializing ---")
w3 = Web3(Web3.HTTPProvider("https://eth-sepolia.public.blastapi.io"))
assert w3.is_connected(), "Failed to connect to Sepolia Ethereum Network"
print("Connected to Sepolia Ethereum Network")
    
abis = load_abis()
print(f"Testing with wallet: {config.WALLET_ADDRESS}")

# 2. Initialize the UniswapPoolHelper
print("\n--- Testing Initialization ---")
try:
    pool_helper = UniswapPoolHelper(w3=w3, pool_address=pool_address, abis=abis)
    print(f"Successfully initialized pool helper for pool: {pool_address}")
    print(f"Token 0: {pool_helper.token0.symbol} ({pool_helper.token0.address})")
    print(f"Token 1: {pool_helper.token1.symbol} ({pool_helper.token1.address})")
    print(f"Pool Fee: {pool_helper.fee}")
except Exception as e:
    print(f"Error initializing pool helper: {e}")

--- Initializing ---
Connected to Sepolia Ethereum Network
Testing with wallet: 0x876C756b92fcc4EB8F634dBe63f3394D97B081ba

--- Testing Initialization ---
Successfully initialized pool helper for pool: 0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1
Token 0: USDC (0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238)
Token 1: WETH (0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14)
Pool Fee: 500


In [3]:
# 3. Test `get_current_price`
print("\n--- Testing get_current_price ---")
try:
    price_t0_in_t1 = pool_helper.get_current_price()
    price_t1_in_t0 = pool_helper.get_current_price(reverse_price=True)
    print(f"Current Price: 1 {pool_helper.token0.symbol} = {price_t0_in_t1:.6f} {pool_helper.token1.symbol}")
    print(f"Current Price: 1 {pool_helper.token1.symbol} = {price_t1_in_t0:.6f} {pool_helper.token0.symbol}")
except Exception as e:
    print(f"Error getting current price: {e}")


--- Testing get_current_price ---
Current Price: 1 USDC = 0.000034 WETH
Current Price: 1 WETH = 29707.395208 USDC


In [4]:
# 4. Test `get_sell_quote` (Selling USDC)
amount_to_sell = Decimal("1000")  # Amount of USDC to sell
print(f"\n--- Testing get_sell_quote (Selling {amount_to_sell} {pool_helper.token0.symbol}) ---")
try:
    token_to_sell = pool_helper.token0 
    token_to_receive = pool_helper.token1

    amount_out, new_price, actual_price, gas_fee = pool_helper.get_sell_quote(
        token_in=token_to_sell,
        amount_in=amount_to_sell,
        quoter_address=config.QUOTER_ADDRESS_SEPOLIA
    )
    print(f"Quote received: Sell {amount_to_sell} {token_to_sell.symbol} to get {amount_out:.8f} {token_to_receive.symbol}")
    print(f"  - Actual Price ({token_to_receive.symbol}/{token_to_sell.symbol}): {actual_price:.8f}")
    print(f"  - New Pool Price after swap: {new_price:.8f}")
    print(f"  - Estimated Gas Fee: {gas_fee:.10f} ETH")
except Exception as e:
    print(f"Error getting sell quote: {e}")


--- Testing get_sell_quote (Selling 1000 USDC) ---
Quote received: Sell 1000 USDC to get 0.03363107 WETH
  - Actual Price (WETH/USDC): 0.00003363
  - New Pool Price after swap: 0.00003363
  - Estimated Gas Fee: 0.0000000905 ETH


In [5]:
# 5. Test `get_sell_quote` (Selling WETH)
amount_to_sell = Decimal("0.01")  # Amount of WETH to sell
print(f"\n--- Testing get_sell_quote (Selling {amount_to_sell} {pool_helper.token1.symbol}) ---")
try:
    token_to_sell = pool_helper.token1
    token_to_receive = pool_helper.token0

    amount_out, new_price, actual_price, gas_fee = pool_helper.get_sell_quote(
        token_in=token_to_sell,
        amount_in=amount_to_sell,
        quoter_address=config.QUOTER_ADDRESS_SEPOLIA
    )
    print(f"Quote received: Sell {amount_to_sell} {token_to_sell.symbol} to get {amount_out:.8f} {token_to_receive.symbol}")
    print(f"  - Actual Price ({token_to_receive.symbol}/{token_to_sell.symbol}): {actual_price:.8f}")
    print(f"  - New Pool Price after swap: {new_price:.8f}")
    print(f"  - Estimated Gas Fee: {gas_fee:.10f} ETH")
except Exception as e:
    print(f"Error getting sell quote: {e}")


--- Testing get_sell_quote (Selling 0.01 WETH) ---
Quote received: Sell 0.01 WETH to get 296.88935000 USDC
  - Actual Price (USDC/WETH): 29688.93500000
  - New Pool Price after swap: 29700.17908891
  - Estimated Gas Fee: 0.0000000934 ETH


In [6]:
# 6. Test `get_buy_quote` (Buying USDC)
amount_to_buy = Decimal("1000")  # Amount of USDC to buy
print(f"\n--- Testing get_buy_quote (Buying {amount_to_buy} {pool_helper.token0.symbol}) ---")
try:
    token_to_buy = pool_helper.token0 
    token_to_pay = pool_helper.token1

    amount_in, new_price, actual_price, gas_fee = pool_helper.get_buy_quote(
        token_out=token_to_buy,
        amount_out=amount_to_buy,
        quoter_address=config.QUOTER_ADDRESS_SEPOLIA
    )
    print(f"Quote received: Buy {amount_to_buy} {token_to_buy.symbol} for {amount_in:.8f} {token_to_pay.symbol}")
    print(f"  - Actual Price ({token_to_pay.symbol}/{token_to_buy.symbol}): {actual_price:.8f}")
    print(f"  - New Pool Price after swap: {new_price:.8f}")
    print(f"  - Estimated Gas Fee: {gas_fee:.10f} ETH")
except Exception as e:
    print(f"Error getting buy quote: {e}")


--- Testing get_buy_quote (Buying 1000 USDC) ---
Quote received: Buy 1000 USDC for 0.03369228 WETH
  - Actual Price (WETH/USDC): 0.00003369
  - New Pool Price after swap: 0.00003369
  - Estimated Gas Fee: 0.0000000943 ETH


In [7]:
# 7. Test `get_buy_quote` (Buying WETH)
amount_to_buy = Decimal("0.0001")  # Amount of WETH to buy
print(f"\n--- Testing get_buy_quote (Buying {amount_to_buy} {pool_helper.token1.symbol}) ---")
try:
    token_to_buy = pool_helper.token1
    token_to_pay = pool_helper.token0

    amount_in, new_price, actual_price, gas_fee = pool_helper.get_buy_quote(
        token_out=token_to_buy,
        amount_out=amount_to_buy,
        quoter_address=config.QUOTER_ADDRESS_SEPOLIA
    )
    print(f"Quote received: Buy {amount_to_buy} {token_to_buy.symbol} for {amount_in:.8f} {token_to_pay.symbol}")
    print(f"  - Actual Price ({token_to_pay.symbol}/{token_to_buy.symbol}): {actual_price:.8f}")
    print(f"  - New Pool Price after swap: {new_price:.8f}")
    print(f"  - Estimated Gas Fee: {gas_fee:.10f} ETH")
except Exception as e:
    print(f"Error getting buy quote: {e}")


--- Testing get_buy_quote (Buying 0.0001 WETH) ---
Quote received: Buy 0.0001 WETH for 2.97223100 USDC
  - Actual Price (USDC/WETH): 29722.31000000
  - New Pool Price after swap: 29707.46741853
  - Estimated Gas Fee: 0.0000000813 ETH


In [8]:
# 8. Test `sell` transaction
# !!! WARNING: This will execute a real transaction on the Sepolia testnet !!!
# !!! It requires your wallet to have a sufficient WETH and ETH (for gas) balance. !!!
print("\n--- Testing `sell` transaction (Commented Out by Default) ---")

amount_to_sell_weth = Decimal("0.00001")  # Amount of WETH to sell

print("This test will execute a REAL transaction. Ensure you are on a testnet and have funds.")
confirm = input("Do you want to proceed with the live sell transaction test? (y/n): ").lower()
if confirm == 'y':
    try:
        # Ensure all required config values are present
        if not all([config.WALLET_ADDRESS, config.WALLET_PRIVATE_KEY]):
            print("Error: WALLET_ADDRESS and WALLET_PRIVATE_KEY must be set in your .env file to execute a transaction.")
            raise ValueError("Missing wallet configuration.")

        slippage = Decimal("0.01") # 1% slippage tolerance

        tx_hash = pool_helper.sell(
            token_in=pool_helper.token1, # Selling WETH
            amount_in=amount_to_sell_weth,
            recipient_address=config.WALLET_ADDRESS,
            private_key=config.WALLET_PRIVATE_KEY,
            slippage=slippage,
            quoter_address=config.QUOTER_ADDRESS_SEPOLIA,
            router_address=config.ROUTER_ADDRESS_SEPOLIA
        )

        if tx_hash:
            print(f"Sell transaction successful! Transaction Hash: 0x{tx_hash}")
            print(f"View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0x{tx_hash}")

    except Exception as e:
        print(f"Error executing sell transaction: {e}")
else:
    print("Live transaction test skipped.")


--- Testing `sell` transaction (Commented Out by Default) ---
This test will execute a REAL transaction. Ensure you are on a testnet and have funds.
Sell transaction successful! Transaction Hash: 0x29e94c6de328ea8e9e3a29f44a1b7b563247afcdede7fd862379c2572edc740c
View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0x29e94c6de328ea8e9e3a29f44a1b7b563247afcdede7fd862379c2572edc740c


In [9]:
# 9. Test `buy` transaction
# !!! WARNING: This will execute a real transaction on the Sepolia testnet !!!
# !!! It requires your wallet to have a sufficient WETH and ETH (for gas) balance. !!!
print("\n--- Testing `buy` transaction (Commented Out by Default) ---")

amount_to_buy_weth = Decimal("0.00001")  # Amount of WETH to buy

print("This test will execute a REAL transaction. Ensure you are on a testnet and have funds.")
confirm = input("Do you want to proceed with the live buy transaction test? (y/n): ").lower()
if confirm == 'y':
    try:
        # Ensure all required config values are present
        if not all([config.WALLET_ADDRESS, config.WALLET_PRIVATE_KEY]):
            print("Error: WALLET_ADDRESS and WALLET_PRIVATE_KEY must be set in your .env file to execute a transaction.")
            raise ValueError("Missing wallet configuration.")

        slippage = Decimal("0.01") # 1% slippage tolerance

        tx_hash = pool_helper.buy(
            token_out=pool_helper.token1, # Buying WETH
            amount_out=amount_to_buy_weth,
            recipient_address=config.WALLET_ADDRESS,
            private_key=config.WALLET_PRIVATE_KEY,
            slippage=slippage,
            quoter_address=config.QUOTER_ADDRESS_SEPOLIA,
            router_address=config.ROUTER_ADDRESS_SEPOLIA
        )

        if tx_hash:
            print(f"Buy transaction successful! Transaction Hash: 0x{tx_hash}")
            print(f"View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0x{tx_hash}")

    except Exception as e:
        print(f"Error executing buy transaction: {e}")
else:
    print("Live transaction test skipped.")


--- Testing `buy` transaction (Commented Out by Default) ---
This test will execute a REAL transaction. Ensure you are on a testnet and have funds.
Buy transaction successful! Transaction Hash: 0xd0af248e81847fe5deaa4da43a965dca9acdef72b4ffd98b37ebe4151a07a51d
View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0xd0af248e81847fe5deaa4da43a965dca9acdef72b4ffd98b37ebe4151a07a51d


In [10]:
# 10. Test `sell` transaction
# !!! WARNING: This will execute a real transaction on the Sepolia testnet !!!
# !!! It requires your wallet to have a sufficient WETH and ETH (for gas) balance. !!!
print("\n--- Testing `sell` transaction (Commented Out by Default) ---")

amount_to_sell_usdc = Decimal("1")  # Amount of USDC to sell

print("This test will execute a REAL transaction. Ensure you are on a testnet and have funds.")
confirm = input("Do you want to proceed with the live sell transaction test? (y/n): ").lower()
if confirm == 'y':
    try:
        # Ensure all required config values are present
        if not all([config.WALLET_ADDRESS, config.WALLET_PRIVATE_KEY]):
            print("Error: WALLET_ADDRESS and WALLET_PRIVATE_KEY must be set in your .env file to execute a transaction.")
            raise ValueError("Missing wallet configuration.")

        slippage = Decimal("0.01") # 1% slippage tolerance

        tx_hash = pool_helper.sell(
            token_in=pool_helper.token0, # Selling USDC
            amount_in=amount_to_sell_usdc,
            recipient_address=config.WALLET_ADDRESS,
            private_key=config.WALLET_PRIVATE_KEY,
            slippage=slippage,
            quoter_address=config.QUOTER_ADDRESS_SEPOLIA,
            router_address=config.ROUTER_ADDRESS_SEPOLIA
        )

        if tx_hash:
            print(f"Sell transaction successful! Transaction Hash: 0x{tx_hash}")
            print(f"View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0x{tx_hash}")

    except Exception as e:
        print(f"Error executing sell transaction: {e}")
else:
    print("Live transaction test skipped.")


--- Testing `sell` transaction (Commented Out by Default) ---
This test will execute a REAL transaction. Ensure you are on a testnet and have funds.
Sell transaction successful! Transaction Hash: 0x86a7aad0d5ec7b0e45d8f4e85c53234f3d5c879a2aec3ddc74a42bed2652d22f
View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0x86a7aad0d5ec7b0e45d8f4e85c53234f3d5c879a2aec3ddc74a42bed2652d22f


In [11]:
# 11. Test `sell` transaction
# !!! WARNING: This will execute a real transaction on the Sepolia testnet !!!
# !!! It requires your wallet to have a sufficient WETH and ETH (for gas) balance. !!!
print("\n--- Testing `buy` transaction (Commented Out by Default) ---")

amount_to_buy_usdc = Decimal("1")  # Amount of USDC to buy

print("This test will execute a REAL transaction. Ensure you are on a testnet and have funds.")
confirm = input("Do you want to proceed with the live buy transaction test? (y/n): ").lower()
if confirm == 'y':
    try:
        # Ensure all required config values are present
        if not all([config.WALLET_ADDRESS, config.WALLET_PRIVATE_KEY]):
            print("Error: WALLET_ADDRESS and WALLET_PRIVATE_KEY must be set in your .env file to execute a transaction.")
            raise ValueError("Missing wallet configuration.")

        slippage = Decimal("0.01") # 1% slippage tolerance

        tx_hash = pool_helper.buy(
            token_out=pool_helper.token0, # Buying USDC
            amount_out=amount_to_buy_usdc,
            recipient_address=config.WALLET_ADDRESS,
            private_key=config.WALLET_PRIVATE_KEY,
            slippage=slippage,
            quoter_address=config.QUOTER_ADDRESS_SEPOLIA,
            router_address=config.ROUTER_ADDRESS_SEPOLIA
        )

        if tx_hash:
            print(f"Buy transaction successful! Transaction Hash: 0x{tx_hash}")
            print(f"View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0x{tx_hash}")

    except Exception as e:
        print(f"Error executing buy transaction: {e}")
else:
    print("Live transaction test skipped.")


--- Testing `buy` transaction (Commented Out by Default) ---
This test will execute a REAL transaction. Ensure you are on a testnet and have funds.
Buy transaction successful! Transaction Hash: 0xaab83f8946df893760e17447293112e05226193974d0fb0abfd0a8856c9ee44e
View on Sepolia Etherscan: https://sepolia.etherscan.io/tx/0xaab83f8946df893760e17447293112e05226193974d0fb0abfd0a8856c9ee44e
